In [29]:


from scipy import ndimage

def sobel_filters(img):
    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)
    
    Ix = ndimage.filters.convolve(img, Kx, int)
    Iy = ndimage.filters.convolve(img, Ky, int)
    
    G = np.hypot(Ix, Iy)
    G = G / G.max() * 255
    theta = np.arctan2(Iy, Ix)
    
    return (G, theta)

def non_max_suppression(img, D):
    M, N = img.shape
    Z = np.zeros((M,N), dtype=np.int8)
    angle = D * 180. / np.pi
    angle[angle < 0] += 180

    
    for i in range(1,M-1):
        for j in range(1,N-1):
            try:
                q = 255
                r = 255
                
               #angle 0
                if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                    q = img[i, j+1]
                    r = img[i, j-1]
                #angle 45
                elif (22.5 <= angle[i,j] < 67.5):
                    q = img[i+1, j-1]
                    r = img[i-1, j+1]
                #angle 90
                elif (67.5 <= angle[i,j] < 112.5):
                    q = img[i+1, j]
                    r = img[i-1, j]
                #angle 135
                elif (112.5 <= angle[i,j] < 157.5):
                    q = img[i-1, j-1]
                    r = img[i+1, j+1]

                if (img[i,j] >= q) and (img[i,j] >= r):
                    Z[i,j] = img[i,j]
                else:
                    Z[i,j] = 0

            except IndexError as e:
                pass
    
    return Z



1.02 s ± 82.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
import numpy as np
import cv2

img = cv2.imread("cropped field 1-1.jpg", cv2.IMREAD_UNCHANGED)
roi2 = img.copy()
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
sobel, angle = sobel_filters(img)
sobel.dtype

dtype('float64')

In [24]:
supress_sobel = non_max_suppression(sobel, angle)

In [25]:
cv2.namedWindow('img',cv2.WINDOW_NORMAL)
cv2.imshow('img', sobel)

cv2.waitKey(0)

-1

In [26]:
cv2.imwrite("supressed.png", supress_sobel)

True

In [27]:
lines = cv2.HoughLinesP(supress_sobel,1,np.pi/180,100,maxLineGap = 100)  # HoughLines function gives an output of a 2d-array containing rho and theta values for the corresponding line equations which crossed the vote limit of 200.  
no_of_lines = len(lines)
print(lines)

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\hough.cpp:471: error: (-215:Assertion failed) image.type() == CV_8UC1 in function 'cv::HoughLinesProbabilistic'


In [ ]:
edges = []
angles= []
lengths = []
for i in range(no_of_lines):
    if ((lines[i][0][3]-lines[i][0][1]) != 0 and (lines[i][0][2]-lines[i][0][0]) != 0):
        ang = abs(math.degrees(math.atan((lines[i][0][3]-lines[i][0][1])/(lines[i][0][2]-lines[i][0][0]))))
        if (abs(90 - ang) <= 5):
            edges.append(lines[i][0])
            angles.append(ang)
            lengths.append(math.sqrt((lines[i][0][2]-lines[i][0][0])**2 + (lines[i][0][3]-lines[i][0][1])**2))
    elif ((lines[i][0][2]-lines[i][0][0]) == 0):
        edges.append(lines[i][0])
        angles.append(90)
        lengths.append(math.sqrt((lines[i][0][2]-lines[i][0][0])**2 + (lines[i][0][3]-lines[i][0][1])**2))
    #else:
        #angles.append(0)
        #lengths.append(sqrt((lines[i][0][2]-lines[i][0][0])**2 + (lines[i][0][3]-lines[i][0][1])**2))
        
temp=angles.copy()
print(edges,angles)

In [ ]:
for i in range(len(edges)):
    cv2.line(roi2, (edges[i][0],edges[i][1]), (edges[i][2],edges[i][3]), (0,255,0), 1)
#cv2.line(roi2, (lines[2][0][0],lines[2][0][1]), (lines[2][0][2],lines[2][0][3]), (0,255,0), 1)

numpy.int8

In [30]:
%%timeit -n 10

import numpy as np
import cv2

img = cv2.imread("cropped field 1-1.jpg", cv2.IMREAD_UNCHANGED)
roi2 = img.copy()
img = cv2.GaussianBlur(img,(3,3),0)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
sobel, angle = sobel_filters(img)

supress_sobel = non_max_suppression(sobel, angle)

971 ms ± 33.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
